Linear Regression 

Librairies Machine Learning 

In [8]:
!pip install findspark 
!pip install pyspark 

In [9]:
import pyspark 
import findspark

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

Importer les données sous Spark 

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TextClassifier").master("local[*]").getOrCreate()

In [12]:
dataset = spark.read.csv('/content/sample_data/BostonHousing.csv',inferSchema=True, header =True)

In [13]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [14]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')

output = assembler.transform(dataset)

# ("Attributes","medv") => (key,value),  key and value cloud be as complex as you need !!!!
finalized_data = output.select("Attributes","medv")

finalized_data.show()

+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



Comme vous pouvez le constater l'input qu'on va soumettre à Spark is a set of (key,value), rappel du cours, (key,value) may be as complex as you need !!!!!, the key is the set of attribute and the value is the TARGET (variable cible)!

Machine Learning 

Diviser dataset en train et test 

In [15]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])

Créer le modèle avec le train_data 

In [16]:
regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)


Afficher l'intercept et le coéffcient h(x) = intercept + coef *x

In [17]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([-0.0994, 0.0346, 0.0384, 2.9472, -18.7867, 4.2879, -0.0033, -1.3076, 0.253, -0.0129, -0.8898, 0.0063, -0.4822])
The Intercept of the model is : 33.513059


Prédiction sur l'ensemble de test 

In [18]:
#To predict the prices on testing set
pred = regressor.evaluate(test_data)

Afficher le résultat 

In [19]:
pred.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.00906,90.0,2.9...|32.2|31.649797119134398|
|[0.0136,75.0,4.0,...|18.9|15.370648796311002|
|[0.01381,80.0,0.4...|50.0| 40.59673186061093|
|[0.01501,90.0,1.2...|50.0|44.836233052674785|
|[0.03502,80.0,4.9...|28.5| 33.03532167939657|
|[0.0351,95.0,2.68...|48.5| 41.62261968153901|
|[0.03584,80.0,3.3...|23.5|29.777329445232272|
|[0.03615,80.0,4.9...|27.9|31.398939872906425|
|[0.03768,80.0,1.5...|34.6| 34.57409977996163|
|[0.03932,0.0,3.41...|22.0| 27.33136045673829|
|[0.04113,25.0,4.8...|28.0|28.652014235935944|
|[0.04544,0.0,3.24...|19.8|21.912990057967626|
|[0.0456,0.0,13.89...|23.3| 26.71267104073081|
|[0.0459,52.5,5.32...|22.3|27.075119127052375|
|[0.04666,80.0,1.5...|30.3| 32.66586381710843|
|[0.05059,0.0,4.49...|23.9|25.196440489005838|
|[0.05497,0.0,5.19...|19.0|21.447867254259833|
|[0.05561,70.0,2.2...|29.0| 31.95135652531899|
|[0.05644,40.

Evaluer le modèle 

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE: 6.025
MSE: 36.295
MAE: 3.796
r2: 0.621
